In [ ]:
import time
import os
from vllm import LLM, SamplingParams
from vllm.assets.audio import AudioAsset

/Users/eyoel/miniconda3/envs/whisper_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-06 09:51:41,743	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
os.environ["VLLM_TORCH_PROFILER_DIR"] = "./vllm_profile"

model = input('model? (tiny, small, medium, ...) > ')

In [ ]:

# Create a Whisper encoder/decoder model instance
llm = LLM(
    model=f"openai/whisper-{model}",
    max_model_len=448,
    max_num_seqs=400,
    limit_mm_per_prompt={"audio": 1},
    kv_cache_dtype="fp8",
)

RuntimeError: Failed to infer device type

In [ ]:
prompts = [
    {
        "prompt": "<|startoftranscript|>",
        "multi_modal_data": {
            "audio": AudioAsset("mary_had_lamb").audio_and_sample_rate,
        },
    },
    {  # Test explicit encoder/decoder prompt
        "encoder_prompt": {
            "prompt": "",
            "multi_modal_data": {
                "audio": AudioAsset("winning_call").audio_and_sample_rate,
            },
        },
        "decoder_prompt": "<|startoftranscript|>",
    }
] 

sampling_params = SamplingParams(
    temperature=0,
    top_p=1.0,
    max_tokens=200,
)

In [ ]:
llm.start_profile()
outputs = llm.generate(prompts, sampling_params)
llm.stop_profile()

In [ ]:
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    encoder_prompt = output.encoder_prompt
    generated_text = output.outputs[0].text
    print(f"\nEncoder prompt: {encoder_prompt!r}, "
          f"\nDecoder prompt: {prompt!r}, "
          f"\nGenerated text: {generated_text!r}")